Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    print(shape)
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

[16, 7, 7, 16]
[10000, 7, 7, 16]
[10000, 7, 7, 16]


In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.610082
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.630847
Minibatch accuracy: 43.8%
Validation accuracy: 49.4%
Minibatch loss at step 100: 1.353469
Minibatch accuracy: 68.8%
Validation accuracy: 54.5%
Minibatch loss at step 150: 0.595978
Minibatch accuracy: 81.2%
Validation accuracy: 74.9%
Minibatch loss at step 200: 0.847285
Minibatch accuracy: 81.2%
Validation accuracy: 77.5%
Minibatch loss at step 250: 1.499088
Minibatch accuracy: 50.0%
Validation accuracy: 77.1%
Minibatch loss at step 300: 0.410731
Minibatch accuracy: 87.5%
Validation accuracy: 79.4%
Minibatch loss at step 350: 0.499878
Minibatch accuracy: 93.8%
Validation accuracy: 77.8%
Minibatch loss at step 400: 0.175184
Minibatch accuracy: 100.0%
Validation accuracy: 80.1%
Minibatch loss at step 450: 0.848838
Minibatch accuracy: 81.2%
Validation accuracy: 78.9%
Minibatch loss at step 500: 0.804762
Minibatch accuracy: 81.2%
Validation accuracy: 81.0%


---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [32]:
image_size = 28
num_channels = 1 # grey scale
num_labels = 10

batch_size = 16
patch_size = 5
depth = 16

stride = 1
pool_stride = 2
pool_kernel_size = 2

num_hidden = 64

num_steps = 2001

graph = tf.Graph()

with graph.as_default():
    
    tf_train_dataset = tf.placeholder(
        dtype=tf.float32,
        shape=(batch_size, image_size, image_size, num_channels)
    )
    
    tf_train_labels = tf.placeholder(
        dtype=tf.float32,
        shape=(batch_size, num_labels)
    )
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    keep_prob = tf.placeholder(dtype=tf.float32)

    global_step = tf.Variable(0, trainable=False)
    initial_learning_rate = 0.05
    learning_rate = tf.train.exponential_decay(
        learning_rate=initial_learning_rate,
        global_step=global_step,
        decay_steps=50,
        decay_rate=0.96,
        staircase=True
    )

    l01_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth],
        stddev=0.1
    ))
    l01_biases = tf.Variable(tf.zeros([depth]))
    
    l02_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth],
        stddev=0.1
    ))
    l02_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    l03_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden],
        stddev=0.1
    ))
    l03_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    l04_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels],
        stddev=0.1
    ))
    l04_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    def model(data):
        conv = tf.nn.conv2d(
            input=data,
            filter=l01_weights,
            strides=[1, stride, stride, 1],
            padding='SAME'
        )
        hidden = tf.nn.relu(conv + l01_biases)
        pool = tf.nn.max_pool(
            value=hidden,
            ksize=[1, pool_kernel_size, pool_kernel_size, 1],
            strides=[1, pool_stride, pool_stride, 1],
            padding='VALID'
        )
        
        conv = tf.nn.conv2d(
            input=pool,
            filter=l02_weights,
            strides=[1, stride, stride, 1],
            padding='SAME'
        )
        hidden = tf.nn.relu(conv + l02_biases)
        pool = tf.nn.max_pool(
            value=hidden,
            ksize=[1, pool_kernel_size, pool_kernel_size, 1],
            strides=[1, pool_stride, pool_stride, 1],
            padding='VALID'
        )
        
        shape = pool.get_shape().as_list()
        reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
        
        hidden = tf.nn.relu(tf.matmul(reshape, l03_weights) + l03_biases)
        dropout = tf.nn.dropout(hidden, keep_prob)
        return tf.matmul(dropout, l04_weights) + l04_biases
    
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))
    
with tf.Session(graph=graph) as session:
    
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_steps):
        
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset+batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset+batch_size), :]
        
        feed = {
            tf_train_dataset: batch_data,
            tf_train_labels: batch_labels,
            keep_prob: 0.5
        }
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed)
        
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval({keep_prob: 1.0}), valid_labels))
            
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval({keep_prob: 1.0}), test_labels))

Initialized
Minibatch loss at step 0: 2.891836
Minibatch accuracy: 12.5%
Validation accuracy: 12.9%
Minibatch loss at step 50: 2.094192
Minibatch accuracy: 31.2%
Validation accuracy: 14.9%
Minibatch loss at step 100: 2.123671
Minibatch accuracy: 18.8%
Validation accuracy: 44.7%
Minibatch loss at step 150: 1.189587
Minibatch accuracy: 50.0%
Validation accuracy: 67.5%
Minibatch loss at step 200: 1.453207
Minibatch accuracy: 50.0%
Validation accuracy: 66.7%
Minibatch loss at step 250: 1.381636
Minibatch accuracy: 62.5%
Validation accuracy: 72.2%
Minibatch loss at step 300: 0.952848
Minibatch accuracy: 68.8%
Validation accuracy: 73.7%
Minibatch loss at step 350: 1.105671
Minibatch accuracy: 68.8%
Validation accuracy: 75.5%
Minibatch loss at step 400: 0.488279
Minibatch accuracy: 87.5%
Validation accuracy: 78.6%
Minibatch loss at step 450: 0.767477
Minibatch accuracy: 75.0%
Validation accuracy: 77.3%
Minibatch loss at step 500: 0.765009
Minibatch accuracy: 81.2%
Validation accuracy: 79.5%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [55]:
image_size = 28
num_channels = 1 # grey scale
num_labels = 10

batch_size = 128
patch_size = 5
depth = 16

stride = 1
pool_stride = 2
pool_kernel_size = 2

num_hidden = 1024

num_steps = 3001

graph = tf.Graph()

with graph.as_default():
    
    tf_train_dataset = tf.placeholder(
        dtype=tf.float32,
        shape=(batch_size, image_size, image_size, num_channels)
    )
    
    tf_train_labels = tf.placeholder(
        dtype=tf.float32,
        shape=(batch_size, num_labels)
    )
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    keep_prob = tf.placeholder(dtype=tf.float32)

    global_step = tf.Variable(0, trainable=False)
    initial_learning_rate = 0.01
    learning_rate = tf.train.exponential_decay(
        learning_rate=initial_learning_rate,
        global_step=global_step,
        decay_steps=50,
        decay_rate=0.9,
        staircase=True
    )

    l01_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth],
        stddev=0.1
    ))
    l01_biases = tf.Variable(tf.zeros([depth]))
    
    l02_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth],
        stddev=0.1
    ))
    l02_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    li01_1_1x1_weights = tf.Variable(tf.truncated_normal(
        [1, 1, 16, 8],
        stddev=0.1
    ))
    li01_1_1x1_biases = tf.Variable(tf.constant(1.0, shape=[8]))
    
    li01_2_1x1_weights = tf.Variable(tf.truncated_normal(
        [1, 1, 16, 8],
        stddev=0.1
    ))
    li01_2_1x1_biases = tf.Variable(tf.constant(1.0, shape=[8]))

    li01_2_3x3_weights = tf.Variable(tf.truncated_normal(
        [3, 3, 8, 16],
        stddev=0.1
    ))
    li01_2_3x3_biases = tf.Variable(tf.constant(1.0, shape=[16]))
    
    li01_3_1x1_weights = tf.Variable(tf.truncated_normal(
        [1, 1, 16, 8],
        stddev=0.1
    ))
    li01_3_1x1_biases = tf.Variable(tf.constant(1.0, shape=[8]))

    li01_3_5x5_weights = tf.Variable(tf.truncated_normal(
        [5, 5, 8, 16],
        stddev=0.1
    ))
    li01_3_5x5_biases = tf.Variable(tf.constant(1.0, shape=[16]))

    li01_4_1x1_weights = tf.Variable(tf.truncated_normal(
        [1, 1, 16, 16],
        stddev=0.1
    ))
    li01_4_1x1_biases = tf.Variable(tf.constant(1.0, shape=[16]))

    li02_1_1x1_weights = tf.Variable(tf.truncated_normal(
        [1, 1, 56, 28],
        stddev=0.1
    ))
    li02_1_1x1_biases = tf.Variable(tf.constant(1.0, shape=[28]))
    
    li02_2_1x1_weights = tf.Variable(tf.truncated_normal(
        [1, 1, 56, 28],
        stddev=0.1
    ))
    li02_2_1x1_biases = tf.Variable(tf.constant(1.0, shape=[28]))

    li02_2_3x3_weights = tf.Variable(tf.truncated_normal(
        [3, 3, 28, 32],
        stddev=0.1
    ))
    li02_2_3x3_biases = tf.Variable(tf.constant(1.0, shape=[32]))
    
    li02_3_1x1_weights = tf.Variable(tf.truncated_normal(
        [1, 1, 56, 28],
        stddev=0.1
    ))
    li02_3_1x1_biases = tf.Variable(tf.constant(1.0, shape=[28]))

    li02_3_5x5_weights = tf.Variable(tf.truncated_normal(
        [5, 5, 28, 32],
        stddev=0.1
    ))
    li02_3_5x5_biases = tf.Variable(tf.constant(1.0, shape=[32]))

    li02_4_1x1_weights = tf.Variable(tf.truncated_normal(
        [1, 1, 56, 32],
        stddev=0.1
    ))
    li02_4_1x1_biases = tf.Variable(tf.constant(1.0, shape=[32]))
    
    l03_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * 124, num_hidden],
        stddev=0.1
    ))
    l03_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    l04_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels],
        stddev=0.1
    ))
    l04_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    def model(data):
        conv = tf.nn.conv2d(
            input=data,
            filter=l01_weights,
            strides=[1, stride, stride, 1],
            padding='SAME'
        )
        hidden = tf.nn.relu(conv + l01_biases)
        pool = tf.nn.max_pool(
            value=hidden,
            ksize=[1, pool_kernel_size, pool_kernel_size, 1],
            strides=[1, pool_stride, pool_stride, 1],
            padding='VALID'
        )
        
        conv = tf.nn.conv2d(
            input=pool,
            filter=l02_weights,
            strides=[1, stride, stride, 1],
            padding='SAME'
        )
        hidden = tf.nn.relu(conv + l02_biases)
        pool = tf.nn.max_pool(
            value=hidden,
            ksize=[1, pool_kernel_size, pool_kernel_size, 1],
            strides=[1, pool_stride, pool_stride, 1],
            padding='VALID'
        )
        
        inception_01_1 = tf.nn.conv2d(
            input=pool,
            filter=li01_1_1x1_weights,
            strides=[1, 1, 1, 1],
            padding='SAME'
        ) + li01_1_1x1_biases
        
        inception_01_2_hidden = tf.nn.conv2d(
            input=pool,
            filter=li01_2_1x1_weights,
            strides=[1, 1, 1, 1],
            padding='SAME'
        ) + li01_2_1x1_biases
        inception_01_2 = tf.nn.conv2d(
            input=tf.nn.relu(inception_01_2_hidden),
            filter=li01_2_3x3_weights,
            strides=[1, 1, 1, 1],
            padding='SAME'
        ) + li01_2_3x3_biases

        inception_01_3_hidden = tf.nn.conv2d(
            input=pool,
            filter=li01_3_1x1_weights,
            strides=[1, 1, 1, 1],
            padding='SAME'
        ) + li01_3_1x1_biases
        inception_01_3 = tf.nn.conv2d(
            input=tf.nn.relu(inception_01_3_hidden),
            filter=li01_3_5x5_weights,
            strides=[1, 1, 1, 1],
            padding='SAME'
        ) + li01_3_5x5_biases
        
        inception_01_4_hidden = tf.nn.max_pool(
            value=pool,
            ksize=[1, pool_kernel_size, pool_kernel_size, 1],
            strides=[1, 1, 1, 1],
            padding='SAME'
        )
        inception_01_4 = tf.nn.conv2d(
            input=inception_01_4_hidden,
            filter=li01_4_1x1_weights,
            strides=[1, 1, 1, 1],
            padding='SAME'
        ) + li01_4_1x1_biases
        
        inception_01 = tf.nn.relu(
            tf.concat(
                values=[inception_01_1, inception_01_2, inception_01_3, inception_01_4],
                axis=3
            )
        )

        inception_02_1 = tf.nn.conv2d(
            input=inception_01,
            filter=li02_1_1x1_weights,
            strides=[1, 1, 1, 1],
            padding='SAME'
        ) + li02_1_1x1_biases
        
        inception_02_2_hidden = tf.nn.conv2d(
            input=inception_01,
            filter=li02_2_1x1_weights,
            strides=[1, 1, 1, 1],
            padding='SAME'
        ) + li02_2_1x1_biases
        inception_02_2 = tf.nn.conv2d(
            input=tf.nn.relu(inception_02_2_hidden),
            filter=li02_2_3x3_weights,
            strides=[1, 1, 1, 1],
            padding='SAME'
        ) + li02_2_3x3_biases

        inception_02_3_hidden = tf.nn.conv2d(
            input=inception_01,
            filter=li02_3_1x1_weights,
            strides=[1, 1, 1, 1],
            padding='SAME'
        ) + li02_3_1x1_biases
        inception_02_3 = tf.nn.conv2d(
            input=tf.nn.relu(inception_02_3_hidden),
            filter=li02_3_5x5_weights,
            strides=[1, 1, 1, 1],
            padding='SAME'
        ) + li02_3_5x5_biases
        
        inception_02_4_hidden = tf.nn.max_pool(
            value=inception_01,
            ksize=[1, pool_kernel_size, pool_kernel_size, 1],
            strides=[1, 1, 1, 1],
            padding='SAME'
        )
        inception_02_4 = tf.nn.conv2d(
            input=inception_02_4_hidden,
            filter=li02_4_1x1_weights,
            strides=[1, 1, 1, 1],
            padding='SAME'
        ) + li02_4_1x1_biases
        
        inception_02 = tf.nn.relu(
            tf.concat(
                values=[inception_02_1, inception_02_2, inception_02_3, inception_02_4],
                axis=3
            )
        )
        
        pool = tf.nn.max_pool(
            value=inception_02,
            ksize=[1, pool_kernel_size, pool_kernel_size, 1],
            strides=[1, 1, 1, 1],
            padding='SAME'
        )

        shape = pool.get_shape().as_list()
        reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
        
        hidden = tf.nn.relu(tf.matmul(reshape, l03_weights) + l03_biases)
        dropout = tf.nn.dropout(hidden, keep_prob)
        return tf.matmul(dropout, l04_weights) + l04_biases
    
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))
    
with tf.Session(graph=graph) as session:
    
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_steps):
        
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset+batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset+batch_size), :]
        
        feed = {
            tf_train_dataset: batch_data,
            tf_train_labels: batch_labels,
            keep_prob: 0.5
        }
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed)
        
        if (step % 100 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval({keep_prob: 1.0}), valid_labels))
            
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval({keep_prob: 1.0}), test_labels))

Initialized
Minibatch loss at step 0: 74.311600
Minibatch accuracy: 11.7%
Validation accuracy: 10.0%
Minibatch loss at step 100: 4.029734
Minibatch accuracy: 8.6%
Validation accuracy: 18.8%
Minibatch loss at step 200: 2.260894
Minibatch accuracy: 33.6%
Validation accuracy: 50.0%
Minibatch loss at step 300: 1.828858
Minibatch accuracy: 45.3%
Validation accuracy: 68.3%
Minibatch loss at step 400: 1.478837
Minibatch accuracy: 58.6%
Validation accuracy: 73.8%
Minibatch loss at step 500: 1.006193
Minibatch accuracy: 69.5%
Validation accuracy: 76.3%
Minibatch loss at step 600: 1.158010
Minibatch accuracy: 69.5%
Validation accuracy: 77.8%
Minibatch loss at step 700: 1.064696
Minibatch accuracy: 71.9%
Validation accuracy: 78.5%
Minibatch loss at step 800: 0.772199
Minibatch accuracy: 76.6%
Validation accuracy: 78.9%
Minibatch loss at step 900: 1.026169
Minibatch accuracy: 65.6%
Validation accuracy: 79.5%
Minibatch loss at step 1000: 0.951023
Minibatch accuracy: 75.8%
Validation accuracy: 79.7%

In [1]:
def TB(cleanup=False):
    import webbrowser
    webbrowser.open('http://127.0.1.1:6006')

    !tensorboard --logdir="logs"

    if cleanup:
        !rm -R logs/

In [2]:
TB()

Starting TensorBoard b'54' at http://Hungs-MBP.Home:6006
(Press CTRL+C to quit)
^C
